<a href="https://colab.research.google.com/github/piabirajdar/AMAZON_REVIEW_RAG_PIPELINE/blob/main/RAGAmazonProductReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle langchain langchain-chroma chromadb sentence-transformers pandas langchain_text_splitters langchain_community langchain_openai

In [ ]:
import kagglehub

# Download latest version of Amazon Fine Food Reviews
path = kagglehub.dataset_download("snap/amazon-fine-food-reviews")

print("Path to dataset files:", path)


In [ ]:
import pandas as pd
df = pd.read_csv(path + "/Reviews.csv")
df = df.dropna(subset=["Text"]).reset_index(drop=True)
print(df)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = splitter.create_documents(df["Text"].tolist())

In [ ]:
print(splits[0])
print(splits[1])

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectordb = Chroma.from_documents(
    splits,
    embedding=embeddings,
    persist_directory="./chroma_reviews"
)
vectordb.persist()


/tmp/ipython-input-3345090399.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Use the following context to answer the question.

CONTEXT:
{context}

QUESTION:
{question}

Answer clearly and concisely.
""")

In [ ]:

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# Build RAG pipeline
rag_chain = (
    {
        "context": retriever,          # fetch relevant docs
        "question": RunnablePassthrough()  # pass question through
    }
    | prompt                           # build final prompt
    | llm                              # call LLM
    | StrOutputParser()                # return clean text
)


In [ ]:
rag_chain.invoke("Summarize negative complaints from the reviews.")

In [ ]:
rag_chain.invoke("What issues do customers repeatedly report in low-rated reviews?")

In [ ]:
rag_chain.invoke("What features do customers appreciate the most?")
